In [2]:
import pandas as pd      # the necessary library
import json              # json duh
from rich import print   # extra python package to giove std output colorful and more readable

## Json Into DataFrame

Ok, so we have done a crash course about `DataFrames` but we haven't scratched the surface (yet). To read much more, but still condensed you can go here: https://pandas.pydata.org/docs/getting_started/intro_tutorials/01_table_oriented.html

For now I want to discuss converting a json file into something a DataFrame can handle. CSV into a dataFrame is really straight forward, but Json is not really setup that way. This isn't always convenient. Most data isn't typically stored this way, but get used to it. Data frames are extremely popular across multiple libraries. Much of your time in the research game will be converting data from on format to another. Whether you are removing anomalies, cleaning up bad values, or altering the format to use for a specific purpose, you will find yourself manipulating data to increase its usability for your purpose. That's what were doing here, taking a somewhat useful Json file and making it fit into a dataFrame. 

Json is `key:value` pairs and not columnar like dataFrames expect. To fix Json data so that a pandas dataFrame can read it we need to turn all of the `key:values` into parallel lists. Remember parallel arrays from early CS? The image below shows how you can use the same index to access values for a single entity. Entity being (in this case) a persons: `Name`, `Age`, and `Sex`.

<img src="./images/parallel_arrays.png" width="400">

It is equivalent to the this Json object (ignoring the empty spaces):
```json
{
    "Name": [
        "Braund, Mr. Owen Harris",
        "Allen, Mr. William Henry",
        "Bonnell, Miss. Elizabeth",
    ],
    "Age": [22, 35, 58],
    "Sex": ["male", "male", "female"],
}
```

When using a Python `dictionary of lists`, the dictionary keys will be used as `column headers` and the values in each list are the values placed the `columns` for the `DataFrame`. Given  the following Json object, can we create a copy that will load into a dataFrame? 

```json
[
    {
        "city": "New York",
        "growth": 4.8,
        "latitude": 40.7127837,
        "longitude": -74.0059413,
        "population": 8405837,
        "rank": 1,
        "state": "New York"
    },
    {
        "city": "Los Angeles",
        "growth": 4.8,
        "latitude": 34.0522342,
        "longitude": -118.2436849,
        "population": 3884307,
        "rank": 2,
        "state": "California"
    },
    {
        "city": "Chicago",
        "growth": -6.1,
        "latitude": 41.8781136,
        "longitude": -87.6297982,
        "population": 2718782,
        "rank": 3,
        "state": "Illinois"
    },
    ...
]
```

This is found [here](./cities_latlon_w_pop.json) in its full form. Let's start by loading the file:

In [8]:
with open("./cities_latlon_w_pop.json") as f:
    cities = json.load(f)

# print first 5 cities
print(cities[:3])

[
    {
        'city': 'New York',
        'growth': 4.8,
        'latitude': 40.7127837,
        'longitude': -74.0059413,
        'population': 8405837,
        'rank': 1,
        'state': 'New York'
    },
    {
        'city': 'Los Angeles',
        'growth': 4.8,
        'latitude': 34.0522342,
        'longitude': -118.2436849,
        'population': 3884307,
        'rank': 2,
        'state': 'California'
    },
    {
        'city': 'Chicago',
        'growth': -6.1,
        'latitude': 41.8781136,
        'longitude': -87.6297982,
        'population': 2718782,
        'rank': 3,
        'state': 'Illinois'
    }
]

We loaded the data, now let's create a new Json object with the seven parallel arrays needed, one for each of the data elements. The keys are `city`, `growth`, ... `state`. I will grab them programmatically, then  create the parallel array version of the json object.


In [10]:
# grab keys from 1st entry
keys = cities[0].keys()

# create a new dictionary (synonymous with json)
cities2 = {}

# iterate over keys and create a list for every key
for key in keys:
    cities2[key] = []

# Note: the keys may not print in the same order you added them. 
# This is ok behavior since dictionaries are not ordered.
print(cities2)

{
    'city': [],
    'growth': [],
    'latitude': [],
    'longitude': [],
    'population': [],
    'rank': [],
    'state': []
}

Now we have the new structure, lets load it up:

In [19]:
for city in cities:
    for key in city:
        cities2[key].append(city[key])

for key in cities2:
    print(cities2[key][:5])


['New York', 'Los Angeles', 'Chicago', 'Houston', 'Philadelphia']

[4.8, 4.8, -6.1, 11, 2.6]

[40.7127837, 34.0522342, 41.8781136, 29.7604267, 39.9525839]

[-74.0059413, -118.2436849, -87.6297982, -95.3698028, -75.1652215]

[8405837, 3884307, 2718782, 2195914, 1553165]

[1, 2, 3, 4, 5]

['New York', 'California', 'Illinois', 'Texas', 'Pennsylvania']

Now we can load it into a dataFrame no problem! 

In [21]:
df = pd.DataFrame(cities2)
print(df)

city  growth   latitude   longitude  population  rank  \
0         New York     4.8  40.712784  -74.005941     8405837     1   
1      Los Angeles     4.8  34.052234 -118.243685     3884307     2   
2          Chicago    -6.1  41.878114  -87.629798     2718782     3   
3          Houston    11.0  29.760427  -95.369803     2195914     4   
4     Philadelphia     2.6  39.952584  -75.165222     1553165     5   
...            ...     ...        ...         ...         ...   ...   
7995  Spanish Fork    78.1  40.114955 -111.654923       36956   998   
7996        Beloit     2.9  42.508348  -89.031776       36888   999   
7997   Panama City     0.1  30.158813  -85.660206       36877  1000   
7998     Anchorage    15.4  61.218056 -149.900278      300950    63   
7999      Honolulu    -6.2  21.306944 -157.858333      347884    54   

             state  
0         New York  
1       California  
2         Illinois  
3            Texas  
4     Pennsylvania  
...            ...  
7995          Utah  
7996     Wisconsin  
7997       Florida  
7998        Alaska  
7999        Hawaii  

[8000 rows x 7 columns]